In [3]:
import pandas as pd
import sys
sys.path.append('.')  # Добавляем текущую директорию в путь поиска модулей
from utils import rec_system

# user_id=666
# user_type=1

test = rec_system(pd.DataFrame({'Type': [1]}))
test


/Users/andrey_kiseleq/Documents/vs_code/itmo/rec_systems/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

divide by zero encountered in matmul

/Users/andrey_kiseleq/Documents/vs_code/itmo/rec_systems/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

overflow encountered in matmul

/Users/andrey_kiseleq/Documents/vs_code/itmo/rec_systems/.venv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

invalid value encountered in matmul



,id,price,level,rooms,area,kitchen_area,geo_lat,geo_lon
0,66,6500000,8,-1,24.80,0.0,55.521764,37.354620
1,234,6500000,10,-1,27.00,0.0,55.967260,37.153689
2,3621,6558909,7,1,24.61,0.0,55.792997,37.740690
3,615,6655460,8,-1,27.40,0.0,55.881954,37.501923
4,54,6304116,9,-1,25.10,0.0,55.418613,37.399485
5,595,6411876,10,-1,27.77,0.0,55.754047,37.620405
6,183,6337495,8,-1,28.50,0.0,55.567659,37.484608
7,1208,6500000,10,1,23.60,0.0,55.623345,37.304879
8,86,6408379,10,-1,23.10,0.0,55.498436,37.509206
9,3623,6686836,7,1,25.09,0.0,55.792997,37.740690


In [1]:
import requests
import pandas as pd

def recommendation_system(data):
    url = 'http://127.0.0.1:500/predict_model'

    # Отправка POST-запроса с данными в формате форм-данных
    response = requests.post(url, json=data)

    # Проверка статуса ответа
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": f"Request failed with status code {response.status_code}"}

# Пример данных для получения рекоммендаций
data = {
    "type": 1
}

# Получение рекоммендаций
recommendations = recommendation_system(data)
recommendations

/Users/andrey_kiseleq/Documents/vs_code/itmo/rec_systems/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[{'id': 183,
  'price': 6337495,
  'level': 8,
  'rooms': -1,
  'area': 28.5,
  'kitchen_area': 0.0,
  'geo_lat': 55.5676585,
  'geo_lon': 37.484608},
 {'id': 595,
  'price': 6411876,
  'level': 10,
  'rooms': -1,
  'area': 27.77,
  'kitchen_area': 0.0,
  'geo_lat': 55.7540471,
  'geo_lon': 37.620405},
 {'id': 234,
  'price': 6500000,
  'level': 10,
  'rooms': -1,
  'area': 27.0,
  'kitchen_area': 0.0,
  'geo_lat': 55.9672604,
  'geo_lon': 37.1536893},
 {'id': 54,
  'price': 6304116,
  'level': 9,
  'rooms': -1,
  'area': 25.1,
  'kitchen_area': 0.0,
  'geo_lat': 55.4186128,
  'geo_lon': 37.3994854},
 {'id': 66,
  'price': 6500000,
  'level': 8,
  'rooms': -1,
  'area': 24.8,
  'kitchen_area': 0.0,
  'geo_lat': 55.5217644,
  'geo_lon': 37.3546197},
 {'id': 615,
  'price': 6655460,
  'level': 8,
  'rooms': -1,
  'area': 27.4,
  'kitchen_area': 0.0,
  'geo_lat': 55.881954,
  'geo_lon': 37.501923},
 {'id': 3621,
  'price': 6558909,
  'level': 7,
  'rooms': 1,
  'area': 24.61,
  'kitchen_a

In [19]:
# Создадим структуру для хранения результатов тестов
stat_test_results = {}

# Выполнение t-теста для каждого пользователя
for user in final_recommendations.keys():
    interacted_obj_ids = user_actions[user_actions['user_id'] == user]['obj_id']
    recommended_obj_ids = final_recommendations[user]

    interacted_objects = homes_fixed.loc[homes_fixed.index.intersection(interacted_obj_ids)]
    recommended_objects = homes_fixed.loc[homes_fixed.index.intersection(recommended_obj_ids)]

    if not interacted_objects.empty and not recommended_objects.empty:
        # Средние значения по объектам
        interacted_mean = interacted_objects.mean(axis=0)
        recommended_mean = recommended_objects.mean(axis=0)

        # Проведение t-теста
        t_stat, p_val = ttest_ind(interacted_mean, recommended_mean, equal_var=False, nan_policy='omit')

        # Сохранение результатов
        stat_test_results[user] = {'t_stat': t_stat, 'p_val': p_val}

        # Вывод результатов для каждого пользователя
        print(f'User {user}: t_stat = {t_stat:.5f}, p_val = {p_val:.5f}')

# Преобразуем в DataFrame для удобного анализа
stat_test_results_df = pd.DataFrame.from_dict(stat_test_results, orient='index')

# Подсчет доли пользователей, у которых p_val > 0.05
no_significant_diff_ratio = (stat_test_results_df['p_val'] > 0.05).mean()
print(f'Доля пользователей, у которых нет статистически значимой разницы: {no_significant_diff_ratio:.2%}')


User 666: t_stat = -0.00778, p_val = 0.99390
Доля пользователей, у которых нет статистически значимой разницы: 100.00%


если p_val > 0.05, то нет оснований полагать, что есть стат. значимая разница => рекомендации были хорошими для всех юзеров